In [ ]:
import os
os.getcwd()

base = os.getcwd()
print(base)
print(os.listdir(base))
#현재 작업 브랜치 어디에서 작업하고 있는지 위치 확인한다.

c:\Users\heclo\KDT_RE_5th_Project\01_DataAnalysis
['00_energy.ipynb', 'data', 'docs', 'energy_pr1.ipynb', 'energy_pr2.ipynb', 'energy_pr3.ipynb', 'energy_pr4.ipynb', 'preprocessing.py', 'Project Plan', 'ReadMe.md', 'sample.ipynb', '__pycache__']


In [ ]:
os.listdir("./data/processed")
#현 디렉토리에서 data-processed 폴더 찾는다. 

['강원도_월별강수량_통합.xlsx',
 '강원도_월별기온_통합.xlsx',
 '강원도_월별풍속_통합.xlsx',
 '강원도_월일조일사_통합.xlsx',
 '신재생에너지_보급통계_발전량(강원도)_(2021-2023).xlsx',
 '신재생에너지_보급통계_발전량(광역)_(2021-2023).xlsx',
 '한국에너지공단_기초지자체별 신재생에너지 보급 현황.xlsx']

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import preprocessing as pre

In [11]:
dfs = pd.read_excel('./data/processed/강원도_월일조일사_통합.xlsx', sheet_name=None)
dfs["강원도(강릉)"].head(10)

,\t\t지점번호,지점명,일시,일조합(hr),일조율(%),일사합(MJ/m2)
0,105,강릉,2020-01,156.4,51.04,238.15
1,105,강릉,2020-02,196.1,62.63,344.96
2,105,강릉,2020-03,251.2,67.62,520.09
3,105,강릉,2020-04,229.5,57.95,570.37
4,105,강릉,2020-05,207.4,47.01,618.57
5,105,강릉,2020-06,212.7,48.05,620.74
6,105,강릉,2020-07,125.9,28.04,453.47
7,105,강릉,2020-08,113.3,26.90,388.74
8,105,강릉,2020-09,139.5,37.43,390.85
9,105,강릉,2020-10,203.3,58.52,407.19


In [12]:
#시트 키 확인 df.keys()
list(dfs.keys())

['강원도(강릉)',
 '강원도(대관령)',
 '강원도(동해)',
 '강원도(북강릉)',
 '강원도(북춘천)',
 '강원도(속초)',
 '강원도(영월)',
 '강원도(원주)',
 '강원도(인제)',
 '강원도(정선군)',
 '강원도(철원)',
 '강원도(춘천)',
 '강원도(태백)',
 '강원도(홍천)']

In [15]:
#결측치 확인
print('전체 데이터 건수:',len(dfs["강원도(정선군)"]))
print('컬럼별 결측치 개수')
len(dfs["강원도(정선군)"]) - dfs["강원도(정선군)"].count()

전체 데이터 건수: 72
컬럼별 결측치 개수


\t\t지점번호       0
지점명            0
일시             0
일조합(hr)        0
일조율(%)         0
일사합(MJ/m2)    70
dtype: int64

In [ ]:
# 시트별로 결측치 훑어보니 특정 시트에서 측정되지 않은 지점이 많아 데이터로서 쓸 수 없고
# 태양광에서 일조량(양)보다 일사량(시간)에 더 영향을 받으므로
# 일사합 자료는 제외하기로 함.
# 일조시간은 가조시간(해가 동쪽에 떠서 서쪽으로 질 때 = 낮의 길이)보다 짧다.
# 낮의 길이는 같은 위도 상에서 동일하게 적용되니 같은 위도에서는 평야지가 산골짜기보다 일조시간이 많다.

In [17]:
processed_dfs = {}
#빈 딕셔너리 만드는 건 시트별로 전처리된 DataFrame을 저장하기 위해 사용.

use_cols = [
    '지점번호',
    '지점명',
    '일시',
    '일조합(hr)',
    '일조율(%)',
]

print("총 시트 수:", len(dfs))

for sheet, df in dfs.items():
    print("처리 중:", sheet) #엑셀 시트가 처리될 때마다 출력

    df.columns = df.columns.str.strip().str.replace(" ", "") #컬럼 명 정리
    #특정 컬럼만 추려내서 데이터화 -원본 dfs 살리기 위해 copy 씀.
    df_use = df[[c for c in use_cols if c in df.columns]].copy() #컬럼 
    processed_dfs[sheet] = df_use
    #전처리 된 df_use를 sheet라는 키로 저장해라.

    # 결측치 처리
    for col in ["일조합(hr)","일조율(%)"]:
            if col in df_use.columns:
                df_use = pre.handle_missing(df_use, col)
    processed_dfs[sheet] = df_use

processed_dfs["강원도(태백)"]

총 시트 수: 14
처리 중: 강원도(강릉)
처리 중: 강원도(대관령)
처리 중: 강원도(동해)
처리 중: 강원도(북강릉)
처리 중: 강원도(북춘천)
처리 중: 강원도(속초)
처리 중: 강원도(영월)
처리 중: 강원도(원주)
처리 중: 강원도(인제)
처리 중: 강원도(정선군)
처리 중: 강원도(철원)
처리 중: 강원도(춘천)
처리 중: 강원도(태백)
처리 중: 강원도(홍천)


,지점번호,지점명,일시,일조합(hr),일조율(%)
0,216,태백,2020-01,120.2,39.01
1,216,태백,2020-02,160.6,51.13
2,216,태백,2020-03,212.6,57.23
3,216,태백,2020-04,222.0,56.16
4,216,태백,2020-05,223.8,50.93
...,...,...,...,...,...
67,216,태백,2025-08,169.4,40.30
68,216,태백,2025-09,83.3,22.34
69,216,태백,2025-10,65.1,18.68
70,216,태백,2025-11,173.4,56.61


In [18]:
#데이터는 이미 월 평균으로 집계된 자료이므로, 결측치는 전체 기간 평균 값으로 대체하였다.

#결측치가 없는지 확인
len(processed_dfs["강원도(강릉)"]) - processed_dfs["강원도(강릉)"].count()

지점번호       0
지점명        0
일시         0
일조합(hr)    0
일조율(%)     0
dtype: int64

In [19]:
output_path = "./data/processed/New_강원도_월별일조일사.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    for sheet, df in processed_dfs.items():
        df.to_excel(
            writer,
            sheet_name=sheet,
            index=False
        )